De code is zodanig geschreven dat je van bovenaf moet beginnen om alle definities, benamingen,... mee te hebben in de latere code.
Maar ook dat er maar 1 plaats is waar je tabelnamen moet aanpassen.

de parameters dienen om de code sneller te laten lopen als je gewoon wilt testen omdat je dan ook nog steeds de hele code moet laten lopen.
Je kan dit in principe vermijden door elke keer uit de SQL tabellen te gaan lezen, maar dat maakt de code ook enorm, enorm traag

In [1]:
%run Fnc_StartupFabricSempy

StatementMeta(, c3a0a556-2fa4-497e-bf43-df432fd9db0a, 9, Finished, Available, Finished)

In [ ]:
prm_starttime = datetime.now()
NotebookName = "Nbt_Landing_GetItems"

StatementMeta(, c3a0a556-2fa4-497e-bf43-df432fd9db0a, 10, Finished, Available, Finished)

In [2]:
Table_Name = 'Landing_Fabric_Items'

StatementMeta(, c3a0a556-2fa4-497e-bf43-df432fd9db0a, 11, Finished, Available, Finished)

In [7]:
lh_abfs_path = fnc_startUp(Table_Name)

StatementMeta(, c3a0a556-2fa4-497e-bf43-df432fd9db0a, 12, Finished, Available, Finished)

niet leeg


SynapseWidget(Synapse.DataFrame, 0be0a703-c648-4801-813a-da091fdcba5e)

In [8]:
#fill list of workspaces
#workspaces = fabric.list_workspaces().sort_values(by='Name', ascending=True)
#df = pd.DataFrame(workspaces)

Workspaces = spark.sql("""select Id, Name
from Lkh_SemanticLink.Landing_Fabric_Workspaces""")


StatementMeta(, c3a0a556-2fa4-497e-bf43-df432fd9db0a, 13, Finished, Available, Finished)

In [ ]:
for Id, Name in Workspaces.toLocalIterator():
    temp_items = fabric.list_items(workspace=Id)
    itemdf = pd.DataFrame(temp_items)
    if not itemdf.empty: # check if the list is not empty to avoid errors
        #prepare items and write them away
        try:
            itemdf = fnc_PrepareColumns(itemdf)
            sparkdf = spark.createDataFrame(itemdf)
            sparkdf = sparkdf.withColumn('WSID', lit(Id))
            sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")
            #print(Table_Name_Items, "created at :", f"{lh_abfs_path}/Tables/{Table_Name_Items}")                        
        except Exception as e:
            print(f"Error fetching Workspace objects for {Name}: {e}")
            continue

StatementMeta(, c3a0a556-2fa4-497e-bf43-df432fd9db0a, 14, Finished, Available, Finished)

In [9]:

# loop through the workspaces
for label, content in df.items():
    if label == 'Id': #loop through the ID's
        for id in content:
            #lijst vullen met alle items in de werkruimte
            temp_items = fabric.list_items(workspace=id)
            itemdf = pd.DataFrame(temp_items)
            if not itemdf.empty: # check if the list is not empty to avoid errors
                #prepare items and write them away
                try:
                    itemdf = fnc_PrepareColumns(itemdf)
                    sparkdf = spark.createDataFrame(itemdf)
                    sparkdf = sparkdf.withColumn('WSID', lit(id))
                    sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")
                    #print(Table_Name_Items, "created at :", f"{lh_abfs_path}/Tables/{Table_Name_Items}")                        
                except Exception as e:
                    print(f"Error fetching Workspace objects for {id}: {e}")
                    continue

StatementMeta(, dcfd2a1b-2cff-4cbd-95e3-80a77d33bca8, 16, Finished, Available, Finished)

In [ ]:
prm_endtime = datetime.utcnow()
fnc_LognotebookRefresh(NotebookName, prm_starttime, prm_endtime)

StatementMeta(, c3a0a556-2fa4-497e-bf43-df432fd9db0a, 16, Finished, Available, Finished)